### ¿Qué genera este script?

Para cada sitio y fecha, el total de usuarios registrados (usuarios.usuariosRegistrados)

In [739]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import re
import random

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [740]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [741]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))

### Definiendo indice principal con fecha de hoy e indice de control

In [742]:
indice = parametros.usuarios_tablero12_index
indice_control = parametros.tableros_mintic_control

### Función para generar JSON compatible con ElactiSearch

In [743]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede, energía, latitud, longitud, COD_ISOid_Beneficiario).

In [744]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob', 'nombreSede' 
                           , 'energiadesc', 'latitud', 'longitud','COD_ISO','id_Beneficiario']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    print("fecha:",now,"- Error en lectura de datos semilla")
    #exit()
    

* Se valida latitud y longitud. Luego se calcula campo location
* Se renombran los campos de semilla

In [745]:
def get_location(x,y='lat'):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None) and (str(x)!=''):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return '4.596389' if y=='lat' else '-74.074639'

In [746]:
datos_semilla['latitud'] = datos_semilla['latitud'].apply(lambda x:get_location(x,'lat'))
datos_semilla['longitud'] = datos_semilla['longitud'].apply(lambda x:get_location(x,'lon'))
datos_semilla['usuarios.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['usuarios.location']=datos_semilla['usuarios.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

datos_semilla = datos_semilla.rename(columns={'lugar_cod' : 'usuarios.centroDigitalUsuarios'
                                            , 'nombre_municipio': 'usuarios.nombreMunicipio'
                                            , 'nombre_departamento' : 'usuarios.nombreDepartamento'
                                            , 'nombre_centro_pob': 'usuarios.localidad'
                                            , 'nombreSede' : 'usuarios.nomCentroDigital'
                                            , 'energiadesc' : 'usuarios.sistemaEnergia'
                                            , 'COD_ISO' : 'usuarios.codISO'
                                            , 'id_Beneficiario' : 'usuarios.idBeneficiario'})
datos_semilla.fillna('', inplace=True)

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [747]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
               "_source": ['usuarios.fechaControl'],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia-tablero12"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['usuarios.fechaControl']
except:
    fecha_ejecucion = '2021-05-01 00:00:00'
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-01 00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

ultima fecha para control de ejecucion: 2021-06-07 00:00:00


### Asociando datos de usuarios

* Se realiza lectura por día procesado
* lugar_cod se usa para cruzar con semilla por site_id
* creado es la fecha de creación del usuario
* tipodoc y documento corresponden a datos del usuario. Se usa para calcular totales

In [748]:
def traeRegistros(fecha_max,fecha_tope):
    total_docs = 10000
    response = es.search(
        index= parametros.ohmyfi_d_u_index,
        body={
                "_source": ['lugar_cod','tipodoc', 'documento', 'creado'],
                "query": {
                  "range": {
                    "creado": {
                      "gte": fecha_max.split(' ')[0]+' 00:00:00',
                      "lt": fecha_tope.split(' ')[0]+' 23:59:59'
                      #"gte": "2021-05-26 10:00:00",
                      #"lt": "2021-05-26 15:00:00"  
                    }
                  }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
#    fields = {}
#    for num, doc in enumerate(elastic_docs):
#        source_data = doc["_source"]
#        for key, val in source_data.items():
#            try:
#                fields[key] = np.append(fields[key], val)
#            except KeyError:
#                fields[key] = np.array([val])

#    return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
    return pd.DataFrame([x["_source"] for x in elastic_docs])

### Se realiza consulta de datos

* Se calcula rango en base a la fecha de control. Para este caso es de un día.
* Se ejecuta la función de consulta con el rango de fechas.
* Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha actual.

In [749]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_registro = traeRegistros(fecha_max_mintic,fecha_tope_mintic)

if datos_registro is None or datos_registro.empty:
    while (datos_registro is None or datos_registro.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        datos_speed = traeRegistros(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

# leyendo indice cambium-devicedevices

De esta formas se asocia las MAC de dispositivos de red INDOOR y OUTDOOR

site_id para cruzar con las misma llave de semilla.
datos del dispositivo: mac, status, ip.
ap_group para identificar si la conexión es indoor/outdoor

In [750]:
total_docs = 30000
try:
    response = es.search(
        index= parametros.cambium_d_d_index,
        body={
                    "_source": ["site_id","mac","status","ip","ap_group"]
                  , "query": {
                    "match_all": {}
                  }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_dev = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    exit()

Se descartan registros con site_id vacios ya que no cruzarán en el merge y se limpian los NaN del dataframe

In [751]:
datos_dev.dropna(subset=['site_id'], inplace=True)
datos_dev.fillna('', inplace=True)
datos_dev = datos_dev.drop(datos_dev[(datos_dev['site_id']=='')].index)

Se corrigen datos de ap group con formato no valido

In [752]:
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("-", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("_", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split(".", n = 1, expand = True)[0]
datos_dev = datos_dev.drop(datos_dev[(datos_dev['ap_group']=='')].index)

Se toman solo los datos con site_id y mac unicos

In [753]:
#datos_dev.drop_duplicates(subset=['site_id','mac'],inplace=True)
datos_dev = datos_dev.drop_duplicates('mac')

Se cambia el nombre a la mac del dispositivo de red para no confundir con la de dispositivos de usuario

In [754]:
datos_dev= datos_dev.rename(columns={'mac' : 'usuarios.macRed','ap_group' : 'usuarios.apGroup'})

# Se lee información de Ohmyfi consumos

Apartir de esta lectura se toma el valor del tiempo promedio de sesión en minutos. Se toma de referencia los campos:

fecha_inicio a partir de la cual se calcula fecha control
tiempo_sesion_minutos
mac_usuario
lugar_cod

In [755]:
def traeSesiones(fecha_max,fecha_tope):
    total_docs = 500000
    response = es.search(
        index= parametros.ohmyfi_consumos_index,
        body={
                  "_source": ["lugar_cod", "tiempo_sesion_minutos","mac_usuario","fecha_inicio"]
                , "query": {
                      "range": {
                            "fecha_inicio": {
                            "gte": fecha_max,
                            "lt": fecha_tope
                            }
                        }
                  }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
    #fields = {}
    #for num, doc in enumerate(elastic_docs):
        #source_data = doc["_source"]
        #for key, val in source_data.items():
            #try:
                #fields[key] = np.append(fields[key], val)
            #except KeyError:
                #fields[key] = np.array([val])
                
    #return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) 

    return pd.DataFrame([x["_source"] for x in elastic_docs])

# Se ejecuta consulta de datos

Se calcula rango en base a la fecha de control. Para este caso es de 10 minutos.
Se ejecuta la función de consulta con el rango de fechas.
Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha y hora actual.

In [756]:
fecha_max_mintic = fecha_ejecucion

fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_consumos = traeSesiones(fecha_max_mintic,fecha_tope_mintic)

if datos_consumos is None or datos_consumos.empty:
    #while (datos_consumos is None or datos_consumos.empty) and ((datetime.strptime(fecha_max_mintic[0:50], '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
    while (datos_consumos is None or datos_consumos.empty) and ((datetime.strptime(fecha_max_mintic[0:50], '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
        datos_consumos = traeSesiones(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

# Total registros  a indice

la lista use_these_keys se usa para referenciar cuales campos del dataframe irán al indice final. si los datos no se declaran en este, no se insertarán

In [757]:
use_these_keys = ['usuarios.fecha'
                  ,'usuarios.siteID'
                  ,'usuarios.usuariosRegistrados'
                  , 'usuarios.anyo'
                  , 'usuarios.mes'
                  , 'usuarios.dia'
                  , 'usuarios.nomCentroDigital'
                  , 'usuarios.codISO'
                  , 'usuarios.localidad'
                  , 'usuarios.nombreDepartamento'
                  , 'usuarios.sistemaEnergia'
                  , 'usuarios.nombreMunicipio'
                  , 'usuarios.idBeneficiario'
                  , 'usuarios.location'
                  , 'usuarios.macRed'
                  , 'usuarios.fechaControl'
                  , 'usuarios.tiempoPromedioSesionSitio'
                  , 'usuarios.apGroup'
                  #, 'usuarios.hora'
                  #, 'usuarios.minuto'
                  , 'ip'
                  , 'status'
                  , 'nombreDepartamento'
                    , 'nombreMunicipio'
                    , 'idBeneficiario'
                    , 'fecha'
                    , 'anyo'
                    , 'mes'
                    , 'dia'
                  , '@timestamp']

def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        document = document.rename(index={ 'usuarios.fecha_x': 'usuarios.fecha','usuarios.anyo_x': 'usuarios.anyo','usuarios.mes_x':'usuarios.mes','usuarios.dia_x':'usuarios.dia','usuarios.nomCentroDigital_x':'usuarios.nomCentroDigital', 'usuarios.codISO_x':'usuarios.codISO', 'usuarios.localidad_x':'usuarios.localidad', 'usuarios.nombreDepartamento_x':'usuarios.nombreDepartamento', 'usuarios.sistemaEnergia_x':'usuarios.sistemaEnergia', 'usuarios.nombreMunicipio_x':'usuarios.nombreMunicipio', 'usuarios.idBeneficiario_x':'usuarios.idBeneficiario', 'usuarios.location_x':'usuarios.location', 'nombreDepartamento_x':'nombreDepartamento', 'nombreMunicipio_x':'nombreMunicipio', 'idBeneficiario_x':'idBeneficiario', 'fecha_x':'fecha', 'anyo_x':'anyo', 'mes_x':'mes', 'dia_x':'dia', '@timestamp_x':'@timestamp'})
        
        print(document)
        #print("=========================")
        yield {
                "_index": indice, 
                "_id": f"{'UsuRegistro-'+ str(document['usuarios.siteID']) + '-' + str(document['usuarios.fecha']) + str(random.randrange(1000))}",
                "_source": filterKeys(document),
            }

Para cada sitio y fecha, se cuentan los documentos de usuario. Esto resulta en el total de usuarios registrados para esa fecha

In [758]:
def invalid_values(df, values=['']):
    return df.loc[df['usuarios.location'].isin(values)]

In [759]:
try:
    
    datos_registro['creado'] = datos_registro['creado'].str[0:10]
    datos_registro['tipo_mas_doc'] = datos_registro['tipodoc'] + datos_registro['documento']
    datos_registro= datos_registro.rename(columns={'creado' : 'usuarios.fecha'
                                                  ,'lugar_cod':'site_id'})

    datos_registro = datos_registro[["usuarios.fecha","site_id","tipo_mas_doc"]].groupby(["usuarios.fecha","site_id"]).agg(['count']).reset_index()
    datos_registro.columns = datos_registro.columns.droplevel(1)
    datos_registro= datos_registro.rename(columns={'tipo_mas_doc' : 'usuarios.usuariosRegistrados'})
    datos_registro["usuarios.anyo"] = datos_registro["usuarios.fecha"].str[0:4]
    datos_registro["usuarios.mes"] = datos_registro["usuarios.fecha"].str[5:7]
    datos_registro["usuarios.dia"] = datos_registro["usuarios.fecha"].str[8:10]
    datos_registro = pd.merge(datos_registro,  datos_semilla, on='site_id', how='inner')
    datos_registro.rename(columns={'site_id': 'usuarios.siteID'
                               }, inplace=True)
    
    datos_registro['nombreDepartamento'] = datos_registro['usuarios.nombreDepartamento']
    datos_registro['nombreMunicipio'] = datos_registro['usuarios.nombreMunicipio']
    datos_registro['idBeneficiario'] = datos_registro['usuarios.idBeneficiario']
    datos_registro['fecha'] = datos_registro['usuarios.fecha']
    datos_registro['anyo'] = datos_registro['usuarios.anyo']
    datos_registro['mes'] = datos_registro['usuarios.mes']
    datos_registro['dia'] = datos_registro['usuarios.dia']
    
    datos_registro['@timestamp'] = now.isoformat()
    #salida = helpers.bulk(es, doc_generator(datos_registro))
    #print("Fecha: ", now,"- Datos Registro usuarios en indice principal:",salida[0])
except Exception as e:
    print("Fecha: ", now,"- Nada que insertar en indice principal",e)

In [760]:
datos_registro.head()

,usuarios.fecha,usuarios.siteID,usuarios.usuariosRegistrados,usuarios.anyo,usuarios.mes,usuarios.dia,usuarios.nomCentroDigital,usuarios.codISO,usuarios.localidad,usuarios.nombreDepartamento,...,usuarios.idBeneficiario,usuarios.location,nombreDepartamento,nombreMunicipio,idBeneficiario,fecha,anyo,mes,dia,@timestamp
0,2021-06-07,10835-ZGYO372,2,2021,06,07,E.U. SIETE DE AGOSTO,CO-ANT,CALLE 115 # 06 - 60,ANTIOQUIA,...,10835,"10.88307108,-75.077732850",ANTIOQUIA,TURBO,10835,2021-06-07,2021,06,07,2021-08-10T01:50:55.892849
1,2021-06-07,11107-ZGYO062,1,2021,06,07,MARIA INMACULADA,CO-ATL,CALLE 15 12 - 20,ATLÁNTICO,...,11107,"10.504888,-74.816395",ATLÁNTICO,BARANOA,11107,2021-06-07,2021,06,07,2021-08-10T01:50:55.892849
2,2021-06-07,11128-ZGYO932,2,2021,06,07,LAS PETRONITAS,CO-ATL,VEREDA LAS PETRONITAS CORDIALIDAD,ATLÁNTICO,...,11128,"8.751218,-75.49306166",ATLÁNTICO,GALAPA,11128,2021-06-07,2021,06,07,2021-08-10T01:50:55.892849
3,2021-06-07,13253-ZGYO527,3,2021,06,07,EL JABO,CO-CES,EL JABO,CESAR,...,13253,"3.985,-75.06861111",CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-10T01:50:55.892849
4,2021-06-07,13258-ZGYO980,15,2021,06,07,IE ANTONIO ENRIQUE DIAZ MARTINEZ,CO-CES,BADILLO,CESAR,...,13258,"10.40971209,-73.5167746",CESAR,VALLEDUPAR,13258,2021-06-07,2021,06,07,2021-08-10T01:50:55.892849


In [761]:
datos_registro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 359 entries, 0 to 358
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   usuarios.fecha                359 non-null    object
 1   usuarios.siteID               359 non-null    object
 2   usuarios.usuariosRegistrados  359 non-null    int64 
 3   usuarios.anyo                 359 non-null    object
 4   usuarios.mes                  359 non-null    object
 5   usuarios.dia                  359 non-null    object
 6   usuarios.nomCentroDigital     359 non-null    object
 7   usuarios.codISO               359 non-null    object
 8   usuarios.localidad            359 non-null    object
 9   usuarios.nombreDepartamento   359 non-null    object
 10  usuarios.sistemaEnergia       359 non-null    object
 11  usuarios.nombreMunicipio      359 non-null    object
 12  usuarios.idBeneficiario       359 non-null    int64 
 13  usuarios.location   

In [762]:
try:
    datos_consumos = datos_consumos.rename(columns={'lugar_cod' :'site_id'})
    datos_consumos['mac_usuario'] = datos_consumos['mac_usuario'].str.replace('-',':')
    datos_consumos['fecha_control'] = datos_consumos["fecha_inicio"].str[0:-4] + '0:00'
    tiempoPromedioSesionSitio=datos_consumos[['site_id','mac_usuario','fecha_control','tiempo_sesion_minutos']].groupby(['site_id','mac_usuario','fecha_control']).agg(['mean']).reset_index()
    tiempoPromedioSesionSitio.columns = tiempoPromedioSesionSitio.columns.droplevel(1)
    tiempoPromedioSesionSitio = tiempoPromedioSesionSitio.rename(columns={'tiempo_sesion_minutos' : 'usuarios.tiempoPromedioSesionSitio'})
    tiempoPromedioSesionSitio['usuarios.tiempoPromedioSesionSitio'] = round(tiempoPromedioSesionSitio['usuarios.tiempoPromedioSesionSitio'],6)
    tiempoPromedioSesionSitio = tiempoPromedioSesionSitio.rename(columns={'mac_usuario' : 'usuarios.macRed'
                                                                     , 'fecha_control' : 'usuarios.fechaControl'})
    tiempoPromedioSesionSitio = pd.merge(tiempoPromedioSesionSitio,datos_semilla, on ='site_id', how='inner')
    tiempoPromedioSesionSitio = pd.merge(tiempoPromedioSesionSitio, datos_dev, on=['site_id','usuarios.macRed'], how='left')
    tiempoPromedioSesionSitio.fillna({'usuarios.apGroup':'No identificado'},inplace=True)
    
    tiempoPromedioSesionSitio = tiempoPromedioSesionSitio.rename(columns={'site_id' : 'usuarios.siteID'})
    tiempoPromedioSesionSitio["usuarios.fecha"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str[0:10]
    tiempoPromedioSesionSitio["usuarios.anyo"] = tiempoPromedioSesionSitio["usuarios.fecha"].str[0:4]
    tiempoPromedioSesionSitio["usuarios.mes"] = tiempoPromedioSesionSitio["usuarios.fecha"].str[5:7]
    tiempoPromedioSesionSitio["usuarios.dia"] = tiempoPromedioSesionSitio["usuarios.fecha"].str[8:10]
    tiempoPromedioSesionSitio["usuarios.hora"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]["usuarios.hora"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    tiempoPromedioSesionSitio["usuarios.minuto"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    
    tiempoPromedioSesionSitio['nombreDepartamento'] = tiempoPromedioSesionSitio['usuarios.nombreDepartamento']
    tiempoPromedioSesionSitio['nombreMunicipio'] = tiempoPromedioSesionSitio['usuarios.nombreMunicipio']
    tiempoPromedioSesionSitio['idBeneficiario'] = tiempoPromedioSesionSitio['usuarios.idBeneficiario']
    tiempoPromedioSesionSitio['fecha'] = tiempoPromedioSesionSitio['usuarios.fecha']
    tiempoPromedioSesionSitio['anyo'] = tiempoPromedioSesionSitio['usuarios.anyo']
    tiempoPromedioSesionSitio['mes'] = tiempoPromedioSesionSitio['usuarios.mes']
    tiempoPromedioSesionSitio['dia'] = tiempoPromedioSesionSitio['usuarios.dia']
    
    tiempoPromedioSesionSitio['@timestamp'] = now.isoformat()
    #salida = helpers.bulk(es, doc_generator(tiempoPromedioSesionSitio))
    #print("Fecha: ", now,"- Tiempo promedio sesion en sitio insertado en indice principal:",salida[0])
except Exception as e:
    print("Fecha: ", now,"- Nada para insertar en indice principal",e)

In [763]:
tiempoPromedioSesionSitio.head()

,usuarios.siteID,usuarios.macRed,usuarios.fechaControl,usuarios.tiempoPromedioSesionSitio,usuarios.nomCentroDigital,usuarios.codISO,usuarios.localidad,usuarios.nombreDepartamento,usuarios.sistemaEnergia,usuarios.nombreMunicipio,...,usuarios.hora,usuarios.minuto,nombreDepartamento,nombreMunicipio,idBeneficiario,fecha,anyo,mes,dia,@timestamp
0,13253-ZGYO527,98:FF:D0:46:AD:16,2021-06-07 00:10:00,5.47,EL JABO,CO-CES,EL JABO,CESAR,RED INTERCONECTADA,VALLEDUPAR,...,00,10,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-10T01:50:55.892849
1,13253-ZGYO527,98:FF:D0:46:AD:16,2021-06-07 00:20:00,4.53,EL JABO,CO-CES,EL JABO,CESAR,RED INTERCONECTADA,VALLEDUPAR,...,00,20,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-10T01:50:55.892849
2,13253-ZGYO527,98:FF:D0:46:AD:16,2021-06-07 00:30:00,12.02,EL JABO,CO-CES,EL JABO,CESAR,RED INTERCONECTADA,VALLEDUPAR,...,00,30,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-10T01:50:55.892849
3,13253-ZGYO527,98:FF:D0:46:AD:16,2021-06-07 00:40:00,3.27,EL JABO,CO-CES,EL JABO,CESAR,RED INTERCONECTADA,VALLEDUPAR,...,00,40,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-10T01:50:55.892849
4,13253-ZGYO527,D6:7A:05:FA:CD:D3,2021-06-07 00:30:00,1.87,EL JABO,CO-CES,EL JABO,CESAR,RED INTERCONECTADA,VALLEDUPAR,...,00,30,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-10T01:50:55.892849


In [764]:
tiempoPromedioSesionSitio.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 326 entries, 0 to 325
Data columns (total 29 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   usuarios.siteID                     326 non-null    object 
 1   usuarios.macRed                     326 non-null    object 
 2   usuarios.fechaControl               326 non-null    object 
 3   usuarios.tiempoPromedioSesionSitio  326 non-null    float64
 4   usuarios.nomCentroDigital           326 non-null    object 
 5   usuarios.codISO                     326 non-null    object 
 6   usuarios.localidad                  326 non-null    object 
 7   usuarios.nombreDepartamento         326 non-null    object 
 8   usuarios.sistemaEnergia             326 non-null    object 
 9   usuarios.nombreMunicipio            326 non-null    object 
 10  usuarios.idBeneficiario             326 non-null    int64  
 11  usuarios.location                   326 non-n

In [765]:
try:
    #result=pd.merge(datos_registro, tiempoPromedioSesionSitio,on=['usuarios.siteID'],how='outer')
    result=pd.merge(datos_registro, tiempoPromedioSesionSitio,on=['usuarios.siteID'],how='inner')
    #salida = helpers.bulk(es, doc_generator(result,use_these_keys))
    result.fillna('', inplace=True)
    #result['usuarios.fechaControl'] = fecha_ejecucion
    result['usuarios.fecha_x'] = fecha_ejecucion[0:4] + '-' + fecha_ejecucion[5:7] + '-' + fecha_ejecucion[8:10]
    #result['usuarios.location_x'] = "0.0000000,0.0000000"
    
    salida = helpers.bulk(es, doc_generator(result))
    print("Fecha: ", now,"- Cantidad de registros insertado en indice principal:",salida[0])
except Exception as e:
    print(e)
    print("Fecha: ", now,"- Nada para insertar en indice principal")

usuarios.fecha                                        2021-06-07
usuarios.siteID                                    13253-ZGYO527
usuarios.usuariosRegistrados                                   3
usuarios.anyo                                               2021
usuarios.mes                                                  06
usuarios.dia                                                  07
usuarios.nomCentroDigital                                EL JABO
usuarios.codISO                                           CO-CES
usuarios.localidad                                       EL JABO
usuarios.nombreDepartamento                                CESAR
usuarios.sistemaEnergia                       RED INTERCONECTADA
usuarios.nombreMunicipio                              VALLEDUPAR
usuarios.idBeneficiario                                    13253
usuarios.location                             3.985,-75.06861111
nombreDepartamento                                         CESAR
nombreMunicipio          

Name: 41, dtype: object
usuarios.fecha                                                               2021-06-07
usuarios.siteID                                                           26489-ZGYO602
usuarios.usuariosRegistrados                                                          8
usuarios.anyo                                                                      2021
usuarios.mes                                                                         06
usuarios.dia                                                                         07
usuarios.nomCentroDigital             INSTITUCION EDUCATIVA SAN JERONIMO - SEDE PRIN...
usuarios.codISO                                                                  CO-CAL
usuarios.localidad                                                         SAN JERONIMO
usuarios.nombreDepartamento                                                      CALDAS
usuarios.sistemaEnergia                                              RED INTERCONECTADA
usuarios

Name: 110, dtype: object
usuarios.fecha                                        2021-06-07
usuarios.siteID                                    31435-ZZZY270
usuarios.usuariosRegistrados                                   4
usuarios.anyo                                               2021
usuarios.mes                                                  06
usuarios.dia                                                  07
usuarios.nomCentroDigital                               SANTA FÉ
usuarios.codISO                                           CO-COR
usuarios.localidad                                      SANTA FE
usuarios.nombreDepartamento                              CÓRDOBA
usuarios.sistemaEnergia                       RED INTERCONECTADA
usuarios.nombreMunicipio                                   CHINÚ
usuarios.idBeneficiario                                    31435
usuarios.location                        9.10290552,-75.39623409
nombreDepartamento                                       CÓRDOBA


usuarios.fecha                                        2021-06-07
usuarios.siteID                                    32004-ZGYO422
usuarios.usuariosRegistrados                                   1
usuarios.anyo                                               2021
usuarios.mes                                                  06
usuarios.dia                                                  07
usuarios.nomCentroDigital                        NUEVA ESPERANZA
usuarios.codISO                                           CO-COR
usuarios.localidad                           VDA NUEVA ESPERANZA
usuarios.nombreDepartamento                              CÓRDOBA
usuarios.sistemaEnergia                       RED INTERCONECTADA
usuarios.nombreMunicipio                            PUEBLO NUEVO
usuarios.idBeneficiario                                    32004
usuarios.location                       8.342140695,-76.09177235
nombreDepartamento                                       CÓRDOBA
nombreMunicipio          

Name: 183, dtype: object
usuarios.fecha                                        2021-06-07
usuarios.siteID                                    32420-ZGYO496
usuarios.usuariosRegistrados                                   3
usuarios.anyo                                               2021
usuarios.mes                                                  06
usuarios.dia                                                  07
usuarios.nomCentroDigital                              LOS CAÑOS
usuarios.codISO                                           CO-COR
usuarios.localidad                                     LOS CAÑOS
usuarios.nombreDepartamento                              CÓRDOBA
usuarios.sistemaEnergia                       RED INTERCONECTADA
usuarios.nombreMunicipio                              SAN CARLOS
usuarios.idBeneficiario                                    32420
usuarios.location                      10.53184983,-73.222004840
nombreDepartamento                                       CÓRDOBA


Name: 239, dtype: object
usuarios.fecha                                              2021-06-07
usuarios.siteID                                          47431-ZZZY243
usuarios.usuariosRegistrados                                         3
usuarios.anyo                                                     2021
usuarios.mes                                                        06
usuarios.dia                                                        07
usuarios.nomCentroDigital             CENTRO EDUCATIVO PIEDRAS BLANCAS
usuarios.codISO                                                 CO-SUC
usuarios.localidad                                     PIEDRAS BLANCAS
usuarios.nombreDepartamento                                      SUCRE
usuarios.sistemaEnergia                             RED INTERCONECTADA
usuarios.nombreMunicipio                                       SAMPUÉS
usuarios.idBeneficiario                                          47431
usuarios.location                                   

Fecha:  2021-08-10 01:50:55.892849 - Cantidad de registros insertado en indice principal: 292


In [766]:
#salida = helpers.bulk(es, doc_generator(result))

In [767]:
result.head()

,usuarios.fecha_x,usuarios.siteID,usuarios.usuariosRegistrados,usuarios.anyo_x,usuarios.mes_x,usuarios.dia_x,usuarios.nomCentroDigital_x,usuarios.codISO_x,usuarios.localidad_x,usuarios.nombreDepartamento_x,...,usuarios.hora,usuarios.minuto,nombreDepartamento_y,nombreMunicipio_y,idBeneficiario_y,fecha_y,anyo_y,mes_y,dia_y,@timestamp_y
0,2021-06-07,13253-ZGYO527,3,2021,06,07,EL JABO,CO-CES,EL JABO,CESAR,...,00,10,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-10T01:50:55.892849
1,2021-06-07,13253-ZGYO527,3,2021,06,07,EL JABO,CO-CES,EL JABO,CESAR,...,00,20,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-10T01:50:55.892849
2,2021-06-07,13253-ZGYO527,3,2021,06,07,EL JABO,CO-CES,EL JABO,CESAR,...,00,30,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-10T01:50:55.892849
3,2021-06-07,13253-ZGYO527,3,2021,06,07,EL JABO,CO-CES,EL JABO,CESAR,...,00,40,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-10T01:50:55.892849
4,2021-06-07,13253-ZGYO527,3,2021,06,07,EL JABO,CO-CES,EL JABO,CESAR,...,00,30,CESAR,VALLEDUPAR,13253,2021-06-07,2021,06,07,2021-08-10T01:50:55.892849


In [768]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292 entries, 0 to 291
Data columns (total 50 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   usuarios.fecha_x                    292 non-null    object 
 1   usuarios.siteID                     292 non-null    object 
 2   usuarios.usuariosRegistrados        292 non-null    int64  
 3   usuarios.anyo_x                     292 non-null    object 
 4   usuarios.mes_x                      292 non-null    object 
 5   usuarios.dia_x                      292 non-null    object 
 6   usuarios.nomCentroDigital_x         292 non-null    object 
 7   usuarios.codISO_x                   292 non-null    object 
 8   usuarios.localidad_x                292 non-null    object 
 9   usuarios.nombreDepartamento_x       292 non-null    object 
 10  usuarios.sistemaEnergia_x           292 non-null    object 
 11  usuarios.nombreMunicipio_x          292 non-n

## Actualiza fecha control

* Se actualiza la fecha de control. Si el calculo supera la fecha actual, se asocia esta ultima.

In [769]:
fecha_ejecucion= (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")[0:15] + '0:00'    

if fecha_ejecucion > str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia-tablero12',
        body = { 'jerarquia-tablero12': 'jerarquia-tablero12','usuarios.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)

actualizada fecha control de ejecucion: 2021-06-07 00:10:00
